In [1]:
import pandas as pd
df = pd.read_csv('data_fin.csv', sep=';')


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100480507 entries, 0 to 100480506
Data columns (total 3 columns):
 #   Column    Dtype  
---  ------    -----  
 0   Cust_Id   int64  
 1   Rating    float64
 2   Movie_Id  int64  
dtypes: float64(1), int64(2)
memory usage: 2.2 GB


In [3]:
#ең жоғары рейтингі (5) бар жазбаларды алып, оларды "Cust_Id" бойынша біріктіреміз
good = df[df['Rating']==5].groupby('Cust_Id')['Movie_Id'].apply(lambda r: ' '.join([str(A) for A in r]))

In [4]:
good.head()


Cust_Id
6     175 457 886 1467 2372 2452 2782 3290 4043 4633...
7     8 30 83 175 257 283 285 313 357 457 458 468 50...
8     1202 1799 1905 2186 3610 3925 4306 5054 5317 5...
10    473 985 1542 1905 2172 3124 3371 3962 4043 430...
25                4432 6786 7605 9326 10643 15107 15270
Name: Movie_Id, dtype: object

In [5]:
!pip install apyori


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5974 sha256=b1e63c03ec510d5394ddf88b9a5fcddd89d4839011f3c9d4286933955229f430
  Stored in directory: c:\users\14ken\appdata\local\pip\cache\wheels\77\3d\a6\d317a6fb32be58a602b1e8c6b5d6f31f79322da554cad2a5ea
Successfully built apyori


In [6]:
import apyori
association_rules = apyori.apriori(good.apply(lambda r: r.split(' ')), 
                                   min_support=0.04, 
                                   min_confidence=0.1, min_lift=2, 
                                   min_length=2)


In [7]:
association_rules

<generator object apriori at 0x00000242A9E5CE00>

In [8]:
# Ассоциативті ережелер сақталатын бос кесте жасаймыз
asr_df = pd.DataFrame(columns = ['from', 'to', 'confidence', 'support', 'lift'])

# Әрбір ассоциативті ережені өңдеу циклі
for item in association_rules:
    
    pair = item[0]  # item[0] — ереженің алғышарты (antecedent), яғни from жағы
    items = [x for x in pair]  # itemset-ті жеке элементтерге бөлеміз
    
    # Кестеге жаңа жол ретінде ереженің сипаттамаларын енгіземіз:
    asr_df.loc[len(asr_df), :] =  ' '.join(list(item[2][0][0])), \  # item[2][0][0] — from (сол жақ), пайдаланушы көрген фильмдер
                                  ' '.join(list(item[2][0][1])), \  # item[2][0][1] — to (оң жақ), ұсынылатын фильм
                                  item[2][0][2], \                  # item[2][0][2] — confidence мәні (сенімділік)
                                  item[1], \                        # item[1] — support мәні (жиі кездесу жиілігі)
                                  item[2][0][3]                     # item[2][0][3] — lift мәні (тәуелділік күшін көрсетеді)


In [9]:
asr_df     #ассоциативті ережелер сақталған DataFrame (яғни кесте)

,from,to,confidence,support,lift
0,10042,10820,0.266215,0.041573,4.110222
1,10042,10947,0.351478,0.054888,2.421943
2,10042,11064,0.348135,0.054366,2.300702
3,10042,11089,0.331616,0.051786,2.488266
4,10042,11283,0.454613,0.070994,2.206234
...,...,...,...,...,...
2018,11521,5582 14961 7057 2452 7230 16265,0.199233,0.041485,4.630662
2019,11521,9628 5582 14961 7057 2452 7230,0.195898,0.04079,4.63469
2020,14240,5582 14961 7057 2452 7230 16265,0.208372,0.041711,4.843073
2021,14240,9628 5582 14961 7057 2452 7230,0.204569,0.04095,4.839831


In [12]:
titles = pd.read_csv(r'C:\Users\14ken\мои работы\data\movie_titles.csv', 
                     encoding='ISO-8859-1',
                     header=None,
                     names=['Movie_Id', 'Year', 'Name'],
                     on_bad_lines='skip')


In [13]:
def get_rule_title(rule):
    # 'from' бағанындағы ID-лерді бөліп, атауларын шығарамыз
    print(titles[titles.Movie_Id.isin(rule['from'].split(' '))]['Name'].values)
    print('----------->')  # бөлгіш
    # 'to' (ұсынылатын фильм) ID арқылы атауын шығарамыз
    print(titles[titles.Movie_Id == int(rule['to'])]['Name'].values)


In [14]:
# функция 99-шы ережеде қандай фильмдер ұсыныс жасағанын және қандай фильм ұсынылғанын көрсетеді.
get_rule_title(asr_df.loc[99])

[]
----------->
['Finding Nemo (Widescreen)']


In [15]:
#ID=159992 болатын кездейсоқ адам үшін ұсыныстар жасауға көшейік
j = 159992
titles[titles.Movie_Id.isin(good.iloc[j].split(' '))]['Name']


Series([], Name: Name, dtype: object)

In [16]:
##Біз кестедегі барлық ережелерді қарап шығып, егер олар адамның көрген фильмдерінде кездессе және олардың рейтингі жоғары болса, бұл ереже оған сәйкес келеді және біз бұл фильмді ұсыныстар тізіміне қоса аламыз.
def print_rule_title(rule):
    return (titles[titles.Movie_Id == int(rule['to'])]['Name'].values)
    

result = []
for A in asr_df.index:
    if len(set(good.iloc[j].split(' ')) & set(asr_df['from'].loc[A].split(' '))) == len(asr_df['from'].loc[A].split(' ')):
        result.append(print_rule_title(asr_df.loc[A])[0])
print(set(result))


{'Pretty Woman', 'Dirty Dancing'}


In [17]:
#Тапсырма 9.4.1.1
import pandas as pd

# Файлды жүктеу (өз жолыңызды көрсетіңіз, егер файл басқа жерде болса)
df = pd.read_csv('data_fin.csv', sep=';')

# Рейтинг 5 болған жазбаларды таңдаймыз
good = df[df['Rating'] == 5]

# 130 ID-і бар пайдаланушының фильмдерін аламыз
user_130_movies = good[good['Cust_Id'] == 130]['Movie_Id'].tolist()

# Нәтижені бір жолға біріктіреміз
result = ' '.join(str(movie_id) for movie_id in user_130_movies)

# Нәтижені шығару
print("130-нөмірлі пайдаланушының ұнатқан фильмдері:")
print(result)


130-нөмірлі пайдаланушының ұнатқан фильмдері:
1865 3456 3962 5515 6029 6428 8159 8327 8782 8784 11165 11242 11701 12084 12232 14482


In [70]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

# 1. Деректерді оқу
# df = pd.read_csv('data_fin.csv', sep=';')  # Егер қайтадан жүктеу қажет болса

# 2. Ең белсенді 500 пайдаланушы мен ең танымал 500 фильм
top_users = df['Cust_Id'].value_counts().head(500).index
top_movies = df['Movie_Id'].value_counts().head(500).index

filtered_df = df[df['Cust_Id'].isin(top_users) & df['Movie_Id'].isin(top_movies)]

# 3. Себет матрицасы (бинарлы түрде)
basket = filtered_df.groupby(['Cust_Id', 'Movie_Id'])['Rating'].count().unstack().fillna(0)

# Бинаризациялау
basket = basket.astype(bool).astype(int)

# 4. Apriori алгоритмі
frequent_itemsets = apriori(basket, min_support=0.05, use_colnames=True)

# 5. Ассоциативті ережелер
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# 6. Индексі 315 ережені шығару
if 315 in rules.index:
    print("Индексі 315 болатын ассоциативті ереже:")
    print(rules.iloc[315])
else:
    print(f"Ережелер саны: {len(rules)}. Индексі 315 табылмады.")


KeyError: 'Cust_Id'

In [23]:
#псырма 9.4.1 3
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# 2. Пайдаланушы-фильм матрицасы
rating_matrix = df.pivot_table(index='Cust_Id', columns='Movie_Id', values='Rating')

# 3. Item-based Collaborative Filtering — фильмдер арасындағы ұқсастық
item_similarity = cosine_similarity(rating_matrix.T.fillna(0))
item_similarity_df = pd.DataFrame(item_similarity, index=rating_matrix.columns, columns=rating_matrix.columns)

# 4. 21-нөмірлі пайдаланушының көрмеген фильмдерін табу
user_ratings = rating_matrix.loc[21]
watched_movies = user_ratings[user_ratings.notna()].index
unwatched_movies = user_ratings[user_ratings.isna()].index

# 5. Болжамды рейтингтерді есептеу
predictions = {}
for movie_id in unwatched_movies:
    sim_scores = item_similarity_df.loc[movie_id, watched_movies]
    user_scores = user_ratings[watched_movies]
    if sim_scores.sum() > 0:
        predicted_rating = (sim_scores @ user_scores) / sim_scores.sum()
        predictions[movie_id] = predicted_rating

# 6. Ұсынылатын фильмдерді сұрыптау
top_recommended = sorted(predictions.items(), key=lambda x: x[1], reverse=True)

# 7. Ұсыныстарды фильм атауларымен біріктіру
recommend_df = pd.DataFrame(top_recommended, columns=['Movie_Id', 'Predicted_Rating'])
merged = recommend_df.merge(movie_titles, on='Movie_Id', how='left')

# 8. Атауы ең қысқа фильмді табу
merged['Title_Length'] = merged['Title'].apply(lambda x: len(str(x)))
shortest_title_movie = merged.sort_values(by='Title_Length').iloc[0]

# 🔚 Нәтиже
print("Пайдаланушы 21 үшін ұсынылған ең қысқа атаулы фильм:")
print(f"🎬 Фильм ID: {shortest_title_movie['Movie_Id']}")
print(f"📌 Атауы: {shortest_title_movie['Title']}")
print(f"⭐ Болжамды рейтинг: {shortest_title_movie['Predicted_Rating']:.2f}")


C:\Users\14ken\anaconda3\Lib\site-packages\pandas\core\reshape\reshape.py:143: RuntimeWarning: overflow encountered in scalar multiply
  num_cells = num_rows * num_columns


ValueError: negative dimensions are not allowed

In [24]:
#Коллаборативті фильтрация
cust_sample = df.Cust_Id.sample(10000)
movie_sample = df.Movie_Id.sample(5000)


In [30]:
import surprise
from surprise import Reader, Dataset
import numpy as np
from collections import defaultdict

# Reader арқылы деректер жиынтығын жүктеу
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(df[df.Cust_Id.isin(cust_sample) &
                              df.Movie_Id.isin(movie_sample)][['Cust_Id', 'Movie_Id', 'Rating']], reader)


In [31]:
#Алгоритмды баптау және модельді оқыту
from surprise import KNNBasic

# sim_options параметрлерін орнату
sim_options = {
    'name': 'cosine',
    'user_based': False  # item-based фильтрация
}

# Модельді баптау
knn = KNNBasic(sim_options=sim_options)
trainingSet = data.build_full_trainset()

# Модельді оқыту
knn.fit(trainingSet)


Computing the cosine similarity matrix...
Done computing similarity matrix.


In [32]:
#Болжау жасау және үздік 3 ұсыныс алу:
# Тест жиынтығын құру
testSet = trainingSet.build_anti_testset()
predictions = knn.test(testSet)

# Үздік 3 ұсынысты алу
def get_top3_recommendations(predictions, topN=3):
    top_recs = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_recs[uid].append((iid, est))
    
    for uid, user_ratings in top_recs.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_recs[uid] = user_ratings[:topN]
    
    return top_recs

top3_recommendations = get_top3_recommendations(predictions)


In [33]:
#Фильмдердің атауларын декодтау және ұсыныстарды шығару:
def print_recs(i):
    for (a, b) in top3_recommendations[i]:
        print(titles[titles.Movie_Id == a]['Name'].values[0], np.round(b, 2))

# Кездейсоқ пайдаланушы үшін ұсыныстар шығару
i = np.random.choice(list(top3_recommendations.keys()))
print_recs(i)


Throne of Blood 4.1
Crumb 4.1
The Rolling Stones: Gimme Shelter 3.95


In [34]:
#Қолданушының қандай фильмдерді жақсы көргенін тексеру:
# Пайдаланушының 5-ке бағалаған фильмдерін табу
films = data.df[(data.df.Cust_Id == i) & (data.df.Rating == 5)]['Movie_Id'].values
titles[titles.Movie_Id.isin(films)]['Name'].values


array(['Seven Samurai', 'Dr. Strangelove', 'Rear Window', 'Annie Hall',
       'Citizen Kane', 'The Bicycle Thief', 'Ran', 'Psycho',
       '2001: A Space Odyssey', 'The Godfather', 'Chinatown',
       'Touch of Evil', 'Suspicion', 'Some Like It Hot'], dtype=object)

In [35]:
#SVD және ALS алгоритмдері Тапсырма 9.7.1
import numpy as np
from scipy.linalg import svd

# Пайдаланушы-фильм матрицасы
user_movie_matrix = np.array([
    [1, 5, 0, 5, 4],
    [5, 4, 4, 3, 2],
    [0, 4, 0, 0, 5],
    [4, 4, 1, 4, 0],
    [0, 4, 3, 5, 0],
    [2, 4, 3, 5, 3]
])

# SVD-ыдырау
U, s, Vt = svd(user_movie_matrix, full_matrices=False)

print("s:", s)


s: [16.46644354  6.21001334  4.39908461  2.90336429  1.58445634]


In [36]:
#Тапсырма 9.7.2. Өлшемді азайту (екі компонент)
# Екі компонентті таңдаймыз
U2 = U[:, :2]
S2 = np.diag(s[:2])
Vt2 = Vt[:2, :]

# №2 пайдаланушы (яғни үшінші жол) бастапқы кеңістікте
user_2 = user_movie_matrix[2]

# lowdim векторын есептейміз (өлшемі азайған пайдаланушы векторы)
user2_lowdim = np.dot(user_2, Vt2.T)

print("user2_lowdim:", user2_lowdim)


user2_lowdim: [-4.09679486  4.68635611]


In [37]:
# Пайдаланушының бағалары векторын қалпына келтіру
user2_predicted = np.dot(user2_lowdim, Vt2)

print("Болжанған бағалар:", user2_predicted)


Болжанған бағалар: [-0.6096677   3.54684982 -0.57140075  1.15887633  4.9116525 ]


In [38]:
#Тапсырма 9.7.3.1
seen = user_movie_matrix[2] != 0
unseen_indices = np.where(~seen)[0]
best_index = unseen_indices[np.argmax(user2_predicted[unseen_indices])]
print("Ең жоғары баға алған көрілмеген фильм индексі:", best_index)


Ең жоғары баға алған көрілмеген фильм индексі: 3


In [39]:
# Тапсырма 9.7.3.2
real = user_movie_matrix[2, 4]
pred = user2_predicted[4]
difference = round(abs(real - pred), 3)
print("Айырмашылық:", difference)


Айырмашылық: 0.088


In [40]:
#4-қадам. Жаңа пайдаланушы мен 3 компонент
# 3 компонентпен SVD
U3 = U[:, :3]
S3 = np.diag(s[:3])
Vt3 = Vt[:3, :]

new_user = np.array([0, 0, 3, 4, 0])

# Жаңа пайдаланушының төмен өлшемді векторы
new_user_lowdim = np.dot(new_user, Vt3.T)

# Кері трансформация
new_user_predicted = np.dot(new_user_lowdim, Vt3)

# Көрмеген фильмдердің индекстері
new_user_seen = new_user != 0
new_unseen_indices = np.where(~new_user_seen)[0]
new_best_index = new_unseen_indices[np.argmax(new_user_predicted[new_unseen_indices])]

print("Жаңа пайдаланушыға ұсынылатын ең үздік фильм индексі:", new_best_index)


Жаңа пайдаланушыға ұсынылатын ең үздік фильм индексі: 1


In [42]:
#Практика. 1. Деректерді жүктеу және шектеу
import pandas as pd
import numpy as np


# Пайдаланушы мен фильм сандары
print("Фильм саны:", df.Movie_Id.nunique())
print("Пайдаланушылар саны:", df.Cust_Id.nunique())

# Тек 10 000 пайдаланушы және 5 000 фильмді алу
cust_sample = df.Cust_Id.sample(10000, random_state=42)
movie_sample = df.Movie_Id.sample(5000, random_state=42)

# Сүзу
df = df[df.Cust_Id.isin(cust_sample) & df.Movie_Id.isin(movie_sample)]


Фильм саны: 17770
Пайдаланушылар саны: 480189


In [47]:
#2. Surprise құрылымы мен SVD оқыту
import surprise
from surprise import Reader, Dataset, SVD

reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(df[['Cust_Id', 'Movie_Id', 'Rating']], reader)

# Оқыту деректерін құру
trainingSet = data.build_full_trainset()
testSet = trainingSet.build_testset()

# Модель жасау және жаттықтыру
svd = SVD()
svd.fit(trainingSet)
predictions = svd.test(testSet)


In [48]:
#3. Үздік 3 ұсынысты табу
from collections import defaultdict

def get_top3_recommendations(predictions, topN=3):
    top_recs = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_recs[uid].append((iid, est))
    for uid, user_ratings in top_recs.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_recs[uid] = user_ratings[:topN]
    return top_recs

top3_recs = get_top3_recommendations(predictions, topN=3)


In [49]:
#4. Фильмдердің атауын қосу
#titles = pd.read_csv('movie_titles.csv', encoding = "ISO-8859-1", 
                     #header = None, names = ['Movie_Id', 'Year', 'Name'])

# Пайдаланушылар тізімінен Кездейсоқ бір пайдаланушыны аламыз
sample_user = list(top3_recs.keys())[0]

# Таңдалған пайдаланушы үшін ұсынылған 3 үздік фильмді шығарамыз
for movie_id, score in top3_recs[sample_user]:
    # Movie_Id арқылы фильм атауын аламыз
    title = titles[titles.Movie_Id == int(movie_id)].Name.values[0]
    
    # Фильм атауы мен болжанған рейтингісін (score) шығарамыз
    print(f"Фильм: {title}, Баға: {score:.2f}")

Фильм: Heathers, Баға: 4.57
Фильм: Lord of the Rings: The Return of the King, Баға: 4.54
Фильм: Lost in Translation, Баға: 4.53


In [52]:
print("Болжамдар саны:", len(predictions))
print("Ұсыныстар саны:", len(top3_recs))


Болжамдар саны: 3799471
Ұсыныстар саны: 9722


In [54]:
#Тапсырма 9.10.1 TrainingSet-тегі пайдаланушылар санын табу:
from surprise.model_selection import train_test_split

# Деректерді оқу
data = surprise.Dataset.load_builtin("ml-100k")
trainset, testset = train_test_split(data, test_size=0.2)

# TrainingSet-тегі пайдаланушылар санын табу
print("Пайдаланушылар саны:", trainset.n_users)


Dataset ml-100k could not be found. Do you want to download it? [Y/n] 

 Y


Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\14ken/.surprise_data/ml-100k
Пайдаланушылар саны: 943


In [55]:
#Тапсырма 9.10.2
#TrainingSet-тегі элементтер (фильмдер) санын табу:
print("Элементтер (фильмдер) саны:", trainset.n_items)


Элементтер (фильмдер) саны: 1650


In [56]:
#Тапсырма 9.10.3
#Барлық рейтингтердің орташа мәнін шығару (жүздікке дейін дөңгелектеп):
mean_rating = round(trainset.global_mean, 2)
print("Орташа рейтинг:", mean_rating)


Орташа рейтинг: 3.53


In [57]:
#Тапсырма 9.10.4
#User-based KNNBasic алгоритмін қолданып testset үшін ұсыныс жасау және 8-ші индекстегі болжамды шығару:
from surprise import KNNBasic
from surprise import Dataset
from surprise.model_selection import train_test_split

# Модельді құру
algo_knn = KNNBasic()
algo_knn.fit(trainset)

# Болжам жасау
predictions_knn = algo_knn.test(testset)

# 8-ші индекстегі ұсынылған фильмнің ID-і
print("8-ші индекстегі фильм ID:", predictions_knn[8].iid)


Computing the msd similarity matrix...
Done computing similarity matrix.
8-ші индекстегі фильм ID: 1192


In [58]:
#Тапсырма 9.10.5
#SVD алгоритмін қолдану, 42-ші индекстегі болжам үшін нақты және бағаланған мән айырмасын есептеу:
from surprise import SVD

# SVD алгоритмін жаттықтыру
algo_svd = SVD(random_state=829)
algo_svd.fit(trainset)

# Болжам жасау
predictions_svd = algo_svd.test(testset)

# 42-ші индекстегі нақты және бағаланған рейтинг айырмасы
diff = abs(predictions_svd[42].r_ui - predictions_svd[42].est)
print("Айырма:", round(diff, 3))


Айырма: 0.576


In [59]:
#Тапсырма 9.10.6
#Екі алгоритм үшін RMSE мәнін есептеу және ең жақсысын табу:
from surprise import accuracy

# RMSE мәндерін есептеу
rmse_knn = accuracy.rmse(predictions_knn)
rmse_svd = accuracy.rmse(predictions_svd)

# Ең жақсы алгоритмді табу
best_rmse = min(rmse_knn, rmse_svd)
print("Ең жақсы RMSE:", round(best_rmse, 2))


RMSE: 0.9896
RMSE: 0.9418
Ең жақсы RMSE: 0.94


In [60]:
#Тапсырма 9.10.7 
df = pd.DataFrame(data.raw_ratings)
df.columns = ['user_id', 'movie_id', 'rating', 3]
df.head()


,user_id,movie_id,rating,3
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [67]:
import pandas as pd

# 4 және одан жоғары бағаланған фильмдер үшін мәтіндік жолды құру
high_rated = df[df['rating'] >= 4]

# Әрбір пайдаланушы үшін фильмдерді жинақтап, мәтіндік жол ретінде көрсету
movie_groups = high_rated.groupby('user_id')['movie_id'].apply(lambda x: ' '.join(map(str, x)))

# user_id 100 үшін жолды шығару
print(movie_groups.iloc[100])



4 435 153 310 258 319 887 211 325 202 8


In [69]:
#Apyori модулінен ассоциативті ережелерді есептеу
from apyori import apriori

# Жоғары баға алған фильмдердің жиынтығын жинақтаймыз
transactions = high_rated.groupby('user_id')['movie_id'].apply(list).values.tolist()

# apriori алгоритмін пайдалану
rules = apriori(transactions, min_support=0.2, min_confidence=0.3, min_lift=2, min_length=2)

# Ережелердің санын шығару
rules_list = list(rules)
print("Ережелер саны:", len(rules_list))


Ережелер саны: 10


In [68]:
#Қай фильм жиі кездеседі?
# Фильмдердің жиілігін есептеу
movie_counts = high_rated['movie_id'].value_counts()

# Ең жиі кездесетін фильм id-ін шығару
most_frequent_movie_id = movie_counts.idxmax()
print("Ең жиі кездесетін фильм id:", most_frequent_movie_id)


Ең жиі кездесетін фильм id: 50
